<a href="https://colab.research.google.com/github/susanth378/Twitter_Sentiment_Analysis/blob/master/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
%tensorflow_version 2.x 
import tensorflow as tf 
from tensorflow import keras
from google.colab import files
import io
import re
import string
from keras.preprocessing.text import  one_hot, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

Using TensorFlow backend.


Importing data from local drive to colab

In [3]:
uploaded_files = files.upload()

Saving Tweets_train.csv to Tweets_train.csv


In [4]:
uploaded_files1 = files.upload()

Saving Tweets_test.csv to Tweets_test.csv


In [0]:
train_df = pd.read_csv(io.BytesIO(uploaded_files['Tweets_train.csv']))
test_df = pd.read_csv(io.BytesIO(uploaded_files1['Tweets_test.csv']))

In [6]:
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [0]:
target = train_df.pop('label')

In [8]:
target.value_counts()

0    29720
1     2242
Name: label, dtype: int64

looking for any null values

In [9]:
train_df.isnull().sum()

id       0
tweet    0
dtype: int64

In [0]:
data = train_df.append(test_df).reset_index()

In [11]:
data.head()

,index,id,tweet
0,0,1,@user when a father is dysfunctional and is s...
1,1,2,@user @user thanks for #lyft credit i can't us...
2,2,3,bihday your majesty
3,3,4,#model i love u take with u all the time in ...
4,4,5,factsguide: society now #motivation


function to remove punctuations number and @user

In [0]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    #print(input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    
    
    return input_txt   

In [0]:
#tweets.column = 'tweet'

In [0]:
data['clean_tweet'] = (np.vectorize(remove_pattern)(data['tweet'], "@[\w]*"))

In [0]:
data['clean_tweet']  = data['clean_tweet'].str.replace("[^a-zA-Z]", " ")
data['clean_tweet'] = data['clean_tweet'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>=3]))

In [0]:
#data['clean_tweet'] = data['clean_tweet'].apply(Punctuation)


In [17]:
data.shape

(49159, 4)

In [18]:
words = set(text_to_word_sequence(' '.join([x for x in data['clean_tweet']])))
vocab_size = len(words)
print(vocab_size)

48386


In [0]:
data['clean_tweet_sequence'] = [one_hot(data['clean_tweet'][i], round(vocab_size)) for i in range(len(data['clean_tweet']))]

In [20]:
data.head()

,index,id,tweet,clean_tweet,clean_tweet_sequence
0,0,1,@user when a father is dysfunctional and is s...,when father dysfunctional and selfish drags hi...,"[31252, 48312, 13952, 26223, 18305, 45136, 380..."
1,1,2,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit can use cause they don ...,"[44870, 18937, 47789, 36000, 46678, 15713, 166..."
2,2,3,bihday your majesty,bihday your majesty,"[22412, 22961, 3583]"
3,3,4,#model i love u take with u all the time in ...,model love take with all the time,"[4174, 40408, 17822, 10319, 34903, 41826, 47289]"
4,4,5,factsguide: society now #motivation,factsguide society now motivation,"[31157, 18523, 11953, 48318]"


In [0]:
train_seq  =  np.array(data['clean_tweet_sequence'])

In [0]:
max_tweet_length = 20 
train_seq = pd.DataFrame(pad_sequences(train_seq, maxlen=max_tweet_length,padding= 'post') )

In [23]:
train_seq.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,31252,48312,13952,26223,18305,45136,38090,45155,3698,38090,33800,45500,0,0,0,0,0,0,0,0
1,44870,18937,47789,36000,46678,15713,16634,4515,42100,20926,17312,18673,25884,4259,40559,0,0,0,0,0
2,22412,22961,3583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4174,40408,17822,10319,34903,41826,47289,0,0,0,0,0,0,0,0,0,0,0,0,0
4,31157,18523,11953,48318,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
test_seq = train_seq[31962:]

In [0]:
train_data = train_seq[:31962]

In [0]:
test_df['sequence'] = data['clean_tweet_sequence'][31962:]

In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
      # recall = TP/ (TP + FN)
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
      # precision = TP/ (TP + FP)
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
x_train,x_val, y_train,  y_val = train_test_split(train_data, target, test_size = 0.3)

In [36]:
embedding_vector_length = 64
model = Sequential() 
model.add(Embedding(vocab_size, embedding_vector_length, input_length=max_tweet_length)) 
model.add(LSTM(100, dropout= 0.5)) 
model.add(Dense(1, activation='sigmoid')) 
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy',f1]) 
print(model.summary()) 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 64)            3096704   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 3,162,805
Trainable params: 3,162,805
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='auto')

In [40]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs= 30, batch_size=256, callbacks= [early_stopping]) 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 22373 samples, validate on 9589 samples
Epoch 1/30
22373/22373 [==============================] - 14s 619us/step - loss: 0.2869 - accuracy: 0.9229 - f1: 0.0220 - val_loss: 0.1444 - val_accuracy: 0.9482 - val_f1: 0.3870
Epoch 2/30
22373/22373 [==============================] - 13s 592us/step - loss: 0.1150 - accuracy: 0.9595 - f1: 0.6647 - val_loss: 0.1145 - val_accuracy: 0.9611 - val_f1: 0.6226
Epoch 3/30
22373/22373 [==============================] - 13s 595us/step - loss: 0.0725 - accuracy: 0.9761 - f1: 0.8188 - val_loss: 0.1172 - val_accuracy: 0.9626 - val_f1: 0.6619
Epoch 4/30
22373/22373 [==============================] - 13s 595us/step - loss: 0.0488 - accuracy: 0.9843 - f1: 0.8822 - val_loss: 0.1235 - val_accuracy: 0.9621 - val_f1: 0.6788
Epoch 5/30
22373/22373 [==============================] - 14s 605us/step - loss: 0.0333 - accuracy: 0.9893 - f1: 0.9242 - val_loss: 0.1383 - val_accuracy: 0.9608 - val_f1: 0.6743
Epoch 6/30
22373/22373 [==============================] 

In [0]:
scores = model.evaluate(x_val, y_val, verbose=0) 

In [42]:
scores

[0.255023618389618, 0.9588069915771484, 0.5361071228981018]

In [0]:
test_set = pd.DataFrame( test_df['id'], columns= ['id'])
test_set['label'] = model.predict_classes(test_seq)

In [62]:
test_set.head()

,id,label
0,31963,0
1,31964,0
2,31965,0
3,31966,0
4,31967,0


In [0]:
test_set.to_excel('tweet_output.xls', index= False)

In [0]:
files.download('tweet_output.xls')